In [18]:
# import libraries we need
import pandas as pd
import numpy as np

## Red Card

In [75]:
# read df
df_wc = pd.read_csv('WorldCups.csv')
df_player = pd.read_csv('WorldCupPlayers.csv')
df_matches = pd.read_csv('WorldCupMatches.csv')

In [76]:
# Remove NaN values to filter easily
df_player = df_player[~df_player['Event'].isna()]
# Keep players with redcard
df_player = df_player.loc[df_player['Event'].str.contains('R')]
df_player = df_player.drop_duplicates()

In [77]:
# Dataframe containing the number of redcard for each team
df_redcard = df_player.groupby('Team Initials').size().reset_index(name='cnt_rc')
df_redcard = df_redcard.rename(columns={'Team Initials':'initials'})
df_redcard

,initials,cnt_rc
0,ALG,2
1,ANG,1
2,ARG,9
3,AUS,4
4,AUT,1
...,...,...
58,URS,2
59,URU,9
60,USA,4
61,YUG,2


In [78]:
# Dataframe containing the number of match for each team
df_cntmatch = (df_matches.groupby('Home Team Initials').size() + df_matches.groupby('Away Team Initials').size()).reset_index(name='cnt_matches')
df_cntmatch = df_cntmatch.rename(columns={'index':'initials'})
df_cntmatch

,initials,cnt_matches
0,ALG,14.0
1,ANG,3.0
2,ARG,81.0
3,AUS,13.0
4,AUT,29.0
...,...,...
77,URU,52.0
78,USA,34.0
79,WAL,5.0
80,YUG,37.0


In [79]:
# Merge the two df and compute the mean number of redcard
df_mean_redcard = pd.merge(df_redcard, df_cntmatch)
df_mean_redcard['mean_redcard'] = df_mean_redcard['cnt_rc'] / df_mean_redcard['cnt_matches']
df_mean_redcard

,initials,cnt_rc,cnt_matches,mean_redcard
0,ALG,2,14.0,0.142857
1,ANG,1,3.0,0.333333
2,ARG,9,81.0,0.111111
3,AUS,4,13.0,0.307692
4,AUT,1,29.0,0.034483
...,...,...,...,...
58,URS,2,31.0,0.064516
59,URU,9,52.0,0.173077
60,USA,4,34.0,0.117647
61,YUG,2,37.0,0.054054


In [80]:
df_mean_redcard[df_mean_redcard['mean_redcard'] == max(df_mean_redcard['mean_redcard'])]

,initials,cnt_rc,cnt_matches,mean_redcard
16,CZE,2,3.0,0.666667
44,SCG,2,3.0,0.666667


In [88]:
df_matches[df_matches['Home Team Initials'] == 'CZE']
df_matches[df_matches['Home Team Initials'] == 'SCG']

,r,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
649,2006,11 Jun 2006 - 15:00,Group C,Zentralstadion,Leipzig,"rn"">Serbia and Montenegro",0,1,Netherlands,,43000.0,0,1,MERK Markus (GER),SCHRAER Christian (GER),SALVER Jan-Hendrik (GER),97410100,97410006,SCG,NED


#### Czech Republic & Serbia and Montenegro	

## La Remontada

In [133]:
# read df
df_wc = pd.read_csv('WorldCups.csv')
df_player = pd.read_csv('WorldCupPlayers.csv')
df_matches = pd.read_csv('WorldCupMatches.csv')

In [134]:
# Function to get the winner
def game_winner(home_goal, home_team, away_goal, away_team):
    if home_goal > away_goal:
        return home_team
    elif home_goal < away_goal:
        return away_team

In [135]:
df_matches['half_time_winner'] = df_matches.apply(lambda x : game_winner(x['Half-time Home Goals'], x['Home Team Initials'], x['Half-time Away Goals'], x['Away Team Initials']),axis=1)
df_matches['half_time_winner'].head()

0     FRA
1     USA
2     YUG
3     ROU
4    None
Name: half_time_winner, dtype: object

In [136]:
df_matches['full_time_winner'] = df_matches.apply(lambda x : game_winner(x['Home Team Goals'], x['Home Team Initials'], x['Away Team Goals'], x['Away Team Initials']),axis=1)
df_matches['full_time_winner'].head()

0    FRA
1    USA
2    YUG
3    ROU
4    ARG
Name: full_time_winner, dtype: object